# Ceneo Scraper

# Biblioteki

In [ ]:
import os
import json
import requests
from bs4 import BeautifulSoup

# Wysyłanie do serwera żądania dostepu do zasobu

In [ ]:
product_id = "94103927"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"

In [ ]:
all_opinions = []
while (url):
    response = requests.get(url)
    page_dom = BeautifulSoup(response.text,"html.parser")
    opinions = page_dom.select("div.js_product-review")

    for opinion  in opinions:
        try:
            single_opinion = {

            "opinion_id" : opinion["data-entry-id"],       
            "author" : opinion.select_one("span.user-post__author-name").text.strip(),
            "recommendation" : opinion.select_one("span.user-post__author-recomendation > em").text.strip(),
            "rating" : opinion.select_one("span.user-post__score-count").text.strip(),
            "content" : opinion.select_one("div.user-post__text").text.strip(),
            "pros" :[p.text.strip() for p in opinion.select("div.review_feature_title-positives ~ div.review-feature_item")],
            "cons" : [c.text.strip() for c in opinion.select("div.review_feature_title-negatives ~ div.review-feature_item")],
            "useful" : opinion.select_one("button.vote-yes > span").text.strip(),
            "useless" : opinion.select_one("button.vote-no > span").text.strip(),
            "publish_date" : opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"].strip(),
            "purchase_date" : opinion.select_one("span.user-post__published > time:nth-child(2)")["datetime"].strip()
            }
            all_opinions.append(single_opinion)
        except (TypeError,AttributeError):
            pass
    try:    
        url = "https://www.ceneo.pl"+page_dom.select_one("a.pagination__next")["href"].strip()
    except TypeError:
        url = None 


##Ekstracja składowych pojedynczej opinii

|Składowa|Selektor|Zmienna|
|--------|--------|-------|
|id opinii|["data-entry-id"]|opinion_id|
|autor|span.user-post__author-name|author|
|rekomendacja|span.user-post__author-recomendation > em|recommendation|
|gwiazdki|span.user-post__score-count|rating|
|treść|div.user-post__text|content|
|lista zalet|div.review_feature_title-positives ~ div.review-feature_item|pros|
|lista wad|div.review_feature_title-negatives ~ div.review-feature_item|cons|
|dla ilu przydatna|button.vote-yes > span|useful|
|dla ilu nieprzydatna|button.vote-no > span|useless|
|data wystawienia|span.user-post__published > time:nth-child(1)["datetime"]|publish_date|
|data zakupu|span.user-post__published > time:nth-child(2)["datetime"]|purchase_date|


In [ ]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")

with open(f"opinions/{product_id}.json", "w", encoding="UTF-8") as jsonfile:
    json.dump(all_opinions, jsonfile, indent =4, ensure_ascii=False)

In [ ]:
print(len(all_opinions))